In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=08ade72e0c9ecdb279397f4cafeee604eca1b384dd3644513ec2119fc7db2357
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
# i = 8
# j = 9
# i + j

In [3]:
import pandas as pd
# import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib
import numpy as np

from sklearn.model_selection import GroupKFold
from surprise.model_selection import PredefinedKFold
# from scipy.stats import permutation_test
from sklearn.model_selection import train_test_split
from surprise import Dataset, SVD
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import cohen_kappa_score, mean_absolute_error , mean_squared_error
# from apyori import apriori
from sklearn.neighbors import NearestNeighbors

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
ratings = pd.read_csv("/kaggle/input/bookrecommender/BX-Book-Ratings.csv",encoding='ISO-8859–1',delimiter=";")
books = pd.read_csv("/kaggle/input/bookrecommender/BX-Books_small.csv",encoding='ISO-8859–1',delimiter=";",on_bad_lines="skip")
users = pd.read_csv("/kaggle/input/bookrecommender/BX-Users.csv",encoding='ISO-8859–1',delimiter=";")

In [5]:
# books['Image-URL-M'].tolist()

In [6]:
from sentence_transformers import SentenceTransformer
sentences = books["Book-Title"].tolist()

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

In [7]:
knn = NearestNeighbors(n_neighbors=10,algorithm='ball_tree',n_jobs=-1)
knn.fit(embeddings)

NearestNeighbors(algorithm='ball_tree', n_jobs=-1, n_neighbors=10)

In [8]:
sentences = ["A concise history of France"]

embeddings = model.encode(sentences)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
bkidx = knn.kneighbors(embeddings, 10, return_distance=True)
score = bkidx[0].tolist()[0]
index = bkidx[1].tolist()[0]

result_knn_df = pd.DataFrame({"score":score,"index":index})
result_knn_df.sort_values(by="score",ascending=False)

,score,index
9,0.865445,29143
8,0.852039,15226
7,0.851584,14606
6,0.846232,21903
5,0.837265,33568
4,0.822894,38275
3,0.802359,42678
2,0.763243,48454
1,0.761287,893
0,0.112639,42985


In [10]:
import pickle

with open('knn.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(knn, file)

In [11]:
books.iloc[result_knn_df['index'].tolist(),:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
42985,0304290009,"A concise history of France,",Marshall B Davidson,1972,Cassell,http://images.amazon.com/images/P/0304290009.0...,http://images.amazon.com/images/P/0304290009.0...,http://images.amazon.com/images/P/0304290009.0...
893,0394559487,Citizens: A Chronicle of the French Revolution,Simon Schama,1989,Random House Inc,http://images.amazon.com/images/P/0394559487.0...,http://images.amazon.com/images/P/0394559487.0...,http://images.amazon.com/images/P/0394559487.0...
48454,0192890697,The French Revolution,"John Morris, Roberts",1978,Oxford Univ Pr (Sd),http://images.amazon.com/images/P/0192890697.0...,http://images.amazon.com/images/P/0192890697.0...,http://images.amazon.com/images/P/0192890697.0...
42678,0520220935,An Encyclopedia of the Wines and Domaines of F...,Clive Coates,2001,University of California Press,http://images.amazon.com/images/P/0520220935.0...,http://images.amazon.com/images/P/0520220935.0...,http://images.amazon.com/images/P/0520220935.0...
38275,0910119791,France,Ronald Throop,2000,Country Press,http://images.amazon.com/images/P/0910119791.0...,http://images.amazon.com/images/P/0910119791.0...,http://images.amazon.com/images/P/0910119791.0...
33568,0863817688,The French Thing: A Novel,Chris Keil,2002,Gwasg Carreg Gwalch,http://images.amazon.com/images/P/0863817688.0...,http://images.amazon.com/images/P/0863817688.0...,http://images.amazon.com/images/P/0863817688.0...
21903,2290053945,Lettres parisiennes : Histoires d'exil,Nancy Huston,1999,J'ai lu,http://images.amazon.com/images/P/2290053945.0...,http://images.amazon.com/images/P/2290053945.0...,http://images.amazon.com/images/P/2290053945.0...
14606,1592400388,Almost French: Love and a New Life in Paris,Sarah Turnbull,2003,Gotham,http://images.amazon.com/images/P/1592400388.0...,http://images.amazon.com/images/P/1592400388.0...,http://images.amazon.com/images/P/1592400388.0...
15226,0679762698,Encore Provence: New Adventures in the South o...,Peter Mayle,2000,Vintage Books USA,http://images.amazon.com/images/P/0679762698.0...,http://images.amazon.com/images/P/0679762698.0...,http://images.amazon.com/images/P/0679762698.0...
29143,0333583191,Francey,Dierdre Purcell,1994,Pan Macmillan,http://images.amazon.com/images/P/0333583191.0...,http://images.amazon.com/images/P/0333583191.0...,http://images.amazon.com/images/P/0333583191.0...
